# nb

> Create FastHTML from a NB

In [11]:
#| default_exp core

In [12]:
#| hide
from nbdev.showdoc import *

In [13]:
%%html
<script type="module" src="https://cdn.jsdelivr.net/npm/zero-md@3?register"></script>
<script src="https://cdn.tailwindcss.com"></script>
<script src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit.min.js"></script>
<script src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit-icons.min.js"></script>
<script type="module" src="https://unpkg.com/franken-wc@0.0.6/dist/js/wc.iife.js"></script>
<link rel="stylesheet" href="https://unpkg.com/franken-wc@0.0.6/dist/css/blue.min.css">

In [14]:
#| export
from fasthtml.common import *
from fasthtml.components import Zero_md
from pathlib import Path
import json
from fh_frankenui.components import *

In [15]:
example_nb_dir = Path('../example_nbs/')
with open(example_nb_dir/'explaining_xt_components.ipynb', 'r') as f: xt_nb = json.load(f)

In [42]:
#| export
def strip_list(l, val='\n'):
    start, end = 0, len(l)
    while start < end and l[start] == val: start += 1
    while end > start and l[end - 1] == val: end -= 1
    return l[start:end]

In [16]:
#| export
def render_md(c):
    return Zero_md(Script(c, type="text/markdown"))

In [17]:
#| export
def render_md_cell(cell):
    assert cell['cell_type'] == 'markdown'
    return Div(cls='md-cell')(render_md(''.join(strip_list(cell['source']))))

In [79]:
# show(Div(*map(render_md_cell,[o for o in xt_nb['cells'][:3] if o['cell_type']=='markdown'])))

In [19]:
#| export
def get_nb_lang(nb): return nb['metadata']['kernelspec']['language']
# get_nb_lang(xt_nb)

In [57]:
#| export
def render_code_source(cell,lang='python'):
    if cell['source']==[]: return ''
    code = f'''```{lang}\n{''.join(strip_list(cell['source']))}'''
    return Div(cls='code-source')(render_md(code))

In [51]:
# show(Div(*map(render_code_source,[o for o in xt_nb['cells'][:10] if o['cell_type']=='code'])))

In [52]:
#| export
def render_code_output(cell,lang='python'):
    res = []
    if len(cell['outputs'])==0: ''
    for output in cell['outputs']:
        if output['output_type'] == 'execute_result':
            data = output['data']
            if 'text/markdown' in data.keys(): res.append(NotStr(''.join(strip_list(data['text/markdown'][1:-1]))))
            elif 'text/plain' in data.keys(): res.append(''.join(strip_list(data['text/plain'])))
        if output['output_type'] == 'stream':
            res.append(''.join(strip_list(output['text'])))
    return Div(cls='code-output')(*res)

In [53]:
nbs_dir = Path('../example_nbs/')
with open(nbs_dir/'00_core.ipynb', 'r') as f: nb = json.load(f)

# render_code_output(nb['cells'][9])

In [54]:
# show(Div(*map(render_code_output,[o for o in xt_nb['cells'][:20] if o['cell_type']=='code'])))

In [70]:
#| export
def render_nb(fpath):
    with open(fpath, 'r') as f: xt_nb = json.load(f)
    fname = Path(fpath).name
    res = []
    for cell in xt_nb['cells']:
        if cell['cell_type']=='code':
            s,o = render_code_source(cell), render_code_output(cell)
            res.append(Card(s,footer=o,cls='mx-20'))
        elif cell['cell_type']=='markdown': res.append(Div(render_md_cell(cell)))
    return Div(cls='mx-20 space-y-6')(*res)

In [71]:
#| hide
import nbdev; nbdev.nbdev_export()